In [29]:
import pandas as pd

path = "C:\\Users\\dhruv lohar\\Downloads\\dogData.xlsx"

df = pd.read_excel(path, sheet_name="Malad Sep 23")

# df["Dog Catching Date"] = pd.to_datetime(df["Dog Catching Date"])

df.dropna(axis=1, how='all', inplace=True)
df.columns = df.columns.str.lower()
df.rename(columns={
    "sr. no ": "caseNumber",
    "dog catching date": "catchingDate",
    "address": "catchingLocation",
    "colour/description  ": "mainColor",
    "surgery date ": "surgeryDate",
    "release date ": "releaseDate",
    "dog age": "age",
    "dog breed": "breed",
    "weight": "dogWeight",
    "skin condition": "SkinCondition",
    "body condition": "BodyCondition",
    "ear notched": "earNotched",
    "arv": "ARV"
}, inplace=True)
df.columns = df.columns.str.replace(" ", "")

df.columns

Index(['caseNumber', 'catchingDate', 'catchingLocation', 'photo', 'mainColor',
       'gender', 'surgeryDate', 'releaseDate', 'age', 'breed', 'dogWeight',
       'BodyCondition', 'agression', 'SkinCondition', 'temperature', 'ARV',
       'procedure', 'earNotched', 'observations', 'day1date',
       'day1observations', 'day2date', 'day2observations', 'day3date',
       'day3observations', 'day4date', 'day4', 'foodintake', 'waterintake'],
      dtype='object')

In [38]:
dogDF = df[["caseNumber", "breed", "mainColor", "gender", "releaseDate", "age", "agression"]].copy()

catcherDF = df[["catchingLocation", "catchingDate"]].copy()

vetDF = df[["dogWeight", "temperature", "SkinCondition", "BodyCondition", "surgeryDate", "procedure", "earNotched", "ARV"]].copy()

caretakerDF = df[["day1date", "day2date", "day3date", "day4date"]].copy()

# dogDF.head(1)
# catcherDF.head(1)
vetDF.head(1)
# caretakerDF.head(1)

,dogWeight,temperature,SkinCondition,BodyCondition,surgeryDate,procedure,earNotched,ARV
0,12.5,100.0,Normal,normal,2023-10-03,OVARIES REMOVED,Yes,Given


In [74]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB URI
db = client["DogNGOApp"]

In [79]:
catcher_coll = db["catchers"]

catcherDF.dropna(inplace=True)
catcherDF['catchingDate'] = pd.to_datetime(catcherDF['catchingDate'], errors='coerce')

catcherData = catcherDF.to_dict(orient="records")

In [82]:
vet_coll = db["doctors"]

vetDF.dropna(inplace=True)
vetDF['surgeryDate'] = pd.to_datetime(vetDF['surgeryDate'], errors='coerce')

vetData = vetDF.to_dict(orient="records")

In [81]:

dogs_collection = db["dogs"]

dogDF.dropna(inplace=True)
dogDF['releaseDate'] = pd.to_datetime(dogDF['releaseDate'], errors='coerce')
dogDF.loc[dogDF['releaseDate'] == 'TBD', 'releaseDate'] = pd.NaT

dogData = dogDF.to_dict(orient="records")

In [95]:
# dogs_collection.delete_many({})
# catcher_coll.delete_many({})
# vet_coll.delete_many({})

dogs_collection.insert_many(dogData)
catcher_coll.insert_many(catcherData)
vet_coll.insert_many(vetData)

InsertManyResult([ObjectId('658343f70f74f81ffe8a1964'), ObjectId('658343f70f74f81ffe8a1965'), ObjectId('658343f70f74f81ffe8a1966'), ObjectId('658343f70f74f81ffe8a1967'), ObjectId('658343f70f74f81ffe8a1968'), ObjectId('658343f70f74f81ffe8a1969'), ObjectId('658343f70f74f81ffe8a196a'), ObjectId('658343f70f74f81ffe8a196b'), ObjectId('658343f70f74f81ffe8a196c'), ObjectId('658343f70f74f81ffe8a196d'), ObjectId('658343f70f74f81ffe8a196e'), ObjectId('658343f70f74f81ffe8a196f'), ObjectId('658343f70f74f81ffe8a1970'), ObjectId('658343f70f74f81ffe8a1971'), ObjectId('658343f70f74f81ffe8a1972'), ObjectId('658343f70f74f81ffe8a1973'), ObjectId('658343f70f74f81ffe8a1974'), ObjectId('658343f70f74f81ffe8a1975'), ObjectId('658343f70f74f81ffe8a1976'), ObjectId('658343f70f74f81ffe8a1977'), ObjectId('658343f70f74f81ffe8a1978'), ObjectId('658343f70f74f81ffe8a1979'), ObjectId('658343f70f74f81ffe8a197a'), ObjectId('658343f70f74f81ffe8a197b'), ObjectId('658343f70f74f81ffe8a197c'), ObjectId('658343f70f74f81ffe8a19

In [96]:
# Fetch all catcher IDs and vet IDs
catcher_ids = [catcher['_id'] for catcher in catcher_coll.find({}, {'_id': 1})]
vet_ids = [vet['_id'] for vet in vet_coll.find({}, {'_id': 1})]

dogs_ids = [dog['_id'] for dog in dogs_collection.find({}, {'_id': 1})]

for dog_id, catcher_id, vet_id in zip(dogs_ids, catcher_ids, vet_ids):
    dogs_collection.update_one(
        {"_id": dog_id},
        {"$set": {
            "catcherDetails": catcher_id,
            "vetDetails": vet_id
        }}
    )

In [100]:
# TESTING IF CORRESPONDS PROPERLY

test_dog = dogs_collection.find_one({'caseNumber': 'JR/OCT/A/080'})
test_catcher = catcher_coll.find_one({'_id': test_dog['catcherDetails']})
test_vet = vet_coll.find_one({'_id': test_dog['vetDetails']})

test_dog
test_catcher
test_vet

{'_id': ObjectId('658343f70f74f81ffe8a19b3'),
 'dogWeight': 17.7,
 'temperature': 102.0,
 'SkinCondition': 'Normal ',
 'BodyCondition': 'normal',
 'surgeryDate': datetime.datetime(2023, 10, 7, 0, 0),
 'procedure': 'OVARIES REMOVED',
 'earNotched': 'Yes ',
 'ARV': 'Given '}